In [1]:
# ==========================================================
# 03_data_processing.ipynb
# Cria objetos PyG e datasets de treino/teste
# ==========================================================

# 0) Project setup (path fix for notebooks)
import sys
from pathlib import Path

# Detect project root (works in notebooks and scripts)
if "__file__" in globals():
    ROOT = Path(__file__).resolve().parents[1]
else:
    ROOT = Path.cwd().parents[0]  # assumes this notebook lives in /notebooks

# Add project root to sys.path so `src` can be imported
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

print(f"📁 Project root detected as: {ROOT.resolve()}")


📁 Project root detected as: /Users/leonardoribeiro/Documents/DataScience/MBA_USP/TCC


In [2]:
# 1) Imports
from pathlib import Path
import pandas as pd

from src.utils import (
    ConfigLoader,
    EnvironmentSetup,
    GraphBuilder,
    DataSplitter,
)


In [3]:
# 2) Load configuration and set environment
cfg = ConfigLoader.load("base.yaml")

seed = cfg.get("general", {}).get("seed", 42)
env = EnvironmentSetup(seed=seed)
print(f"✅ Environment initialized with seed={seed}")


✅ Active device: cpu
GPU detected: None
Torch version: 2.8.0
✅ Environment initialized with seed=42


In [4]:
# 3) Load processed CSVs
data_proc = ROOT / cfg["paths"]["data_processed"]

nodes_with_class_path = data_proc / "elliptic_nodes_with_class.csv"
edges_path            = data_proc / "elliptic_edges.csv"
target_path           = data_proc / "elliptic_target.csv"  # (opcional aqui)

print("🔍 Looking for processed files in:")
print(f"   - {data_proc.resolve()}")

df_nodes_with_class = pd.read_csv(nodes_with_class_path)
df_edges            = pd.read_csv(edges_path)
df_target           = pd.read_csv(target_path)  # opcional

print("✅ Loaded:")
print(f"   - Nodes with class: {df_nodes_with_class.shape}")
print(f"   - Edges:            {df_edges.shape}")
print(f"   - Target:           {df_target.shape}")


🔍 Looking for processed files in:
   - /Users/leonardoribeiro/Documents/DataScience/MBA_USP/TCC/data/processed
✅ Loaded:
   - Nodes with class: (203769, 168)
   - Edges:            (234355, 2)
   - Target:           (203769, 2)


In [5]:
# 4) Build PyG graphs (GraphBuilder)
# All feature columns start with "feature_"
feature_cols = [c for c in df_nodes_with_class.columns if c.startswith("feature_")]

# Destination for elliptic_graphs.pt
output_dir = data_proc

# Optional: read exclusion list from YAML if present
exclude_steps = cfg.get("splits", {}).get("exclude_steps", [])

builder = GraphBuilder(
    df_nodes=df_nodes_with_class,
    df_edges=df_edges,
    feature_cols=feature_cols,
    output_dir=output_dir,
    exclude_steps=exclude_steps,
)

# Build and save PyG graphs
builder.run(filename="elliptic_graphs.pt")
graphs = builder.graphs

print(f"✅ {len(graphs)} graphs created and saved to {(output_dir / 'elliptic_graphs.pt').resolve()}")


✅ Created 49 graphs (time_steps 1–49)
💾 Saved 49 graphs to /Users/leonardoribeiro/Documents/DataScience/MBA_USP/TCC/data/processed/elliptic_graphs.pt
✅ 49 graphs created and saved to /Users/leonardoribeiro/Documents/DataScience/MBA_USP/TCC/data/processed/elliptic_graphs.pt


In [6]:
# 5) Split data into train/test and save CSVs (DataSplitter)
splitter = DataSplitter(cfg, df_nodes_with_class)

# ensure splitter uses absolute data_processed path when running from /notebooks
splitter.data_processed = data_proc.resolve()

df_train, df_test1, df_test2, df_test = splitter.run(save_csv=True)

print("\n✅ Data splitting and CSV export complete!")


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL torch_geometric.data.data.DataEdgeAttr was not an allowed global by default. Please use `torch.serialization.add_safe_globals([torch_geometric.data.data.DataEdgeAttr])` or the `torch.serialization.safe_globals([torch_geometric.data.data.DataEdgeAttr])` context manager to allowlist this global if you trust this class/function.

Check the documentat

In [ ]:
# 6) Quick verification / summary
splits_dir = (data_proc / "splits").resolve()

print("\n📊 Summary:")
print(f" - Train nodes: {len(df_train)}")
print(f" - Test1 nodes: {len(df_test1)}")
print(f" - Test2 nodes: {len(df_test2)}")
print(f" - Total labeled nodes: {len(df_train) + len(df_test1) + len(df_test2)}")
print(f"\n📂 Split CSVs saved under: {splits_dir}")
print("   - train_nodes.csv")
print("   - test1_nodes.csv")
print("   - test2_nodes.csv")

print("\n🚀 Processing completed successfully!")



📊 Summary:
 - Train nodes: 29894
 - Test1 nodes: 9983
 - Test2 nodes: 6687
 - Total labeled nodes: 46564

📂 Split CSVs saved under: /Users/leonardoribeiro/Documents/DataScience/MBA_USP/TCC/data/processed/splits
   - train_nodes.csv
   - test1_nodes.csv
   - test2_nodes.csv

🚀 Processing completed successfully!
